In [1]:
import pandas as pd
from tqdm import tqdm
from collections import Counter
import glob
import pickle

In [2]:

#example file:lacI_example.txt

#list fo all the genes we want to invetigate
names=["lacI"]

dici_={}
for gene in tqdm(names):

    rawout=open(gene+"_example.txt")
    rawl=rawout.readlines()
    rawout.close()
    check=0
    all_cds=[]
    # Create empty DataFrame with specified columns
    df = pd.DataFrame(columns=["Beg", "End", "Std", "Total", "CodPot", "StrtSc", "Codon", "RBSMot", "Spacer", "RBSScr", "UpsScr", "TypeScr", "GCCont"])
    
    for l in rawl:
        if l[0] == "#":
            check = 0
            # If we found a hit and the DataFrame has at least one row
            if len(df) >= 1:
                # Filter out rows where "Codon" is "Edge"
                df = df[df["Codon"] != "Edge"]
                if len(df) == 0:
                    continue
                # Here we can do something with the finished DataFrame
                df["Total"] = df["Total"].astype(float)
                df["Len"] = abs(df["Beg"].astype(int) - df["End"].astype(int))
                maxl = max(df["Len"])
                maxv = max(df["Total"])
                # Filter out rows where "Len" is greater than 90% of the maximum length
                df = df[df["Len"] > (maxl * 0.9)]
                # Filter out rows where "Total" is greater than 80% of the maximum value
                df2 = df[df["Total"] > (maxv * 0.8)]
                
                # If the "RBSMot" value at the row with the maximum "Total" is "None"
                if df2.loc[df2["Total"].idxmax(), "RBSMot"] == "None":
                    # Append the normalized value counts of "Codon" to all_cds
                    all_cds.append(df2["Codon"].value_counts(normalize=True))

                # If the "RBSMot" value at the row with the maximum "Total" is not "None"
                if df2.loc[df2["Total"].idxmax(), "RBSMot"] != "None":
                    # Filter out rows where "RBSMot" is "None"
                    df3 = df2[df2["RBSMot"] != "None"]
                    # Append the normalized value counts of "Codon" to all_cds
                    all_cds.append(df3["Codon"].value_counts(normalize=True))
                    
                # Reset the DataFrame for the next iteration
                df = pd.DataFrame(columns=["Beg", "End", "Std", "Total", "CodPot", "StrtSc", "Codon", "RBSMot", "Spacer", "RBSScr", "UpsScr", "TypeScr", "GCCont"])
        
        if check == 1:
            if l[0] != "\n":
                # Add a new row to the DataFrame with values from the current line
                df.loc[len(df), :] = l.strip().split("\t")

        if l[:3] == "Beg":
            check = 1
            
            
            
    #Count total occurences of the three start codons:

    sumatg=0
    sumgtg=0
    sumttg=0
    for x in all_cds:
        try:
            sumatg+=x["ATG"]
        except:
            pass
        try:
            sumgtg+=x["GTG"]
        except:
            pass
        try:
            sumttg+=x["TTG"]
        except:
            pass
    #Create a dictionary to save the results, multiple genes and startcodons possible
    print("Total occurences of start codons in: "+gene)
    print("ATG: "+str(sumatg)+" \t GTG: "+ str(sumgtg)+" \t TTG: "+ str(sumttg))
    dici_[gene]={}
    dici_[gene]["ATG"]=sumatg
    dici_[gene]["GTG"]=sumgtg
    dici_[gene]["TTG"]=sumttg
    

100%|██████████| 1/1 [00:00<00:00, 55.40it/s]

Total occurences of start codons in: lacI
ATG: 0.3333333333333333 	 GTG: 1.6666666666666665 	 TTG: 0


In [3]:
#Optional: saves the dictionary
with open('dici_lacI.pkl', 'wb') as fp:
    pickle.dump(dici_, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [4]:
#list of investigated start codons
codons=["ATG","GTG","TTG"]

In [5]:
#Calculate relative abundance of the start codons over each gene
var=[]
tr=[]
val=[]
for gen in names:
    dc=dici_[gen]
    summa=dc["ATG"]+dc["GTG"]+dc["TTG"]
    for co in codons:
        var.append(co)
        tr.append(gen)
        try:
            val.append(dc[co]/summa)
        except:
            val.append(0.0)

In [9]:
#Create dataframe in a nice format for plotting
newd=pd.DataFrame(columns=["Start Codon","Gene","Percentage"])

newd["Start Codon"]=var
newd["Gene"]=tr
newd["Percentage"]=val

newd

,Start Codon,Gene,Percentage
0,ATG,lacI,0.166667
1,GTG,lacI,0.833333
2,TTG,lacI,0.000000


In [7]:
#Optional: save the dataframe as csv, to continue to plotting or further analysis
df1str=newd.to_csv(sep="\t",index=False)
f= open("R_rel_frequencies_ecoli_v3.csv","w+")
f.write(df1str)
f.close()